# 2. Dataset/DataLoaderの実装

## 2.1. Janomeのインストール

In [1]:
!pip install janome

from janome.tokenizer import Tokenizer
j_t = Tokenizer()

# 単語分割の関数
def tokenizer_janome(text):
    return [token for token in j_t.tokenize(text, wakati=True)]

text = "機械学習が好きです。"
print(tokenizer_janome(text))


     |████████████████████████████████| 21.5MB 1.3MB/s 
['機械', '学習', 'が', '好き', 'です', '。']


## 2.2. 前処理関数の定義

In [8]:
import re

def preprocessing_text(text):
    text = re.sub("\r", "", text)  # 改行の削除
    text = re.sub("\n", "", text)  # 改行の削除
    text = re.sub("　", "", text)  # 全角スペースの削除
    text = re.sub(" ", "", text)   # 半角スペースの削除
    text = re.sub(r"[0-9 ０-９]", "0", text)    # 数字を全て0にする
    return text

text = "昨日は とても暑く、気温が　3６度もあった。"
print(preprocessing_text(text))

def tokenizer_with_preprocessing(text):
    return tokenizer_janome(preprocessing_text(text))

print(tokenizer_with_preprocessing(text))

昨日はとても暑く、気温が00度もあった。
['昨日', 'は', 'とても', '暑く', '、', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


## 2.3. Fieldの定義

In [10]:
import torchtext

max_length = 25 # 1つの文章内の最大単語数

TEXT = torchtext.data.Field(
    sequential=True,    # 可変データか
    tokenize=tokenizer_with_preprocessing,  # 単語分割の関数
    use_vocab=True,     # 単語を辞書に追加するか
    lower=True,         # アルファベットを小文字にするか
    include_lengths=True,   # 単語数を保持するか
    batch_first=True,       # バッチの次元が先にくるか
    fix_length=max_length   # 各文章をパディングして同じ長さにする
)

LABEL = torchtext.data.Field(
    sequential=False,    # 可変データか
    use_vocab=False,     # 単語を辞書に追加するか
)

## 2.4. データの読み込みサンプル

In [24]:
# ※GoogleDriveに接続
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(path="./drive/My Drive/Transformer/data", train="text_train.tsv", validation="text_val.tsv", test="text_test.tsv",
                                     format="tsv", fields=[("Text", TEXT), ("Label", LABEL)])

print(len(train_ds))
print(vars(train_ds[0]))
print(vars(train_ds[1]))
print(train_ds[0].Text)
print(train_ds[0].Label)


4
{'Text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'Label': '0'}
{'Text': ['機械', '学習', 'が', '好き', 'です', '。'], 'Label': '1'}
['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。']
0


## 2.5. 単語ID辞書の作成

In [29]:
TEXT.build_vocab(train_ds, min_freq=1)
# TEXT.vocab.freqs    # 頻度を出力
TEXT.vocab.stoi     # 単語IDの出力



defaultdict(<function torchtext.vocab._default_unk_index>,
            {'0': 18,
             '<pad>': 1,
             '<unk>': 0,
             '、': 7,
             '。': 3,
             'い': 19,
             'いる': 11,
             'か': 12,
             'から': 20,
             'が': 8,
             'し': 9,
             'する': 21,
             'その': 22,
             'た': 23,
             'て': 13,
             'で': 24,
             'です': 25,
             'と': 2,
             'な': 4,
             'に': 26,
             'に対して': 27,
             'の': 5,
             'は': 28,
             'まし': 29,
             'ます': 14,
             'を': 10,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '分類': 15,
             '取り組み': 37,
             '商品': 38,
             '女性': 39,
             '女王': 40,
             '好き': 41,
             '姫': 42,
             '学習': 43,
     

## 2.6. DataLoaderの作成

In [33]:
train_dl = torchtext.data.Iterator(train_ds, batch_size=2, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=2, train=False, sort=False)
test_dl = torchtext.data.Iterator(test_ds, batch_size=2, train=False, sort=False)

batch = next(iter(train_dl))
print(batch.Text)
print(batch.Label)


(tensor([[45, 43,  8, 41, 25,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1],
        [16, 24, 28, 38, 34,  5, 49,  6, 27,  7, 22,  6,  8, 31,  4, 17, 10,  9,
         13, 11,  6,  4,  5, 12,  7]]), tensor([ 6, 25]))
tensor([1, 0])
